# Package Imports and Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import collections
import numpy as np
import scipy as sp
import statsmodels.api as sm


Bad key "text.kerning_factor" on line 4 in
/Users/michaelnowotny/anaconda3/envs/continuous_time_mcmc/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
from divergence import *

# Distributions and Samples

## Construct Artificial Sample from two Normal Distributions

This example considers two different normal distributions $p$ and $q$ with
$p = N(2, 9)$ and $q = N(1, 4)$.

In [4]:
# fix random seed for reproducibility
np.random.seed(42)

# set parameters of the normal distributions p and q
mu_p = 2
sigma_p = 3
mu_q = 1
sigma_q = 2

# draw samples from each normal distribution
n = 10000

def draw_normal(mu, sigma, n: int, antithetic: bool = False):
    z = np.random.randn(n)
    if antithetic: 
        z = np.hstack((z, -z))
    
    return mu + sigma * z

samples_p = draw_normal(mu_p, sigma_p, n=n, antithetic=True)
samples_q = draw_normal(mu_q, sigma_q, n=n, antithetic=True)

# fit a non-parametric density estimate for both distributions
kde_p = sm.nonparametric.KDEUnivariate(samples_p)
kde_q = sm.nonparametric.KDEUnivariate(samples_q)
kde_p.fit()
kde_q.fit()

# construct exact normal densities for p and q
pdf_p = lambda x: sp.stats.norm.pdf(x, mu_p, sigma_p)
pdf_q = lambda x: sp.stats.norm.pdf(x, mu_q, sigma_q)

# compute support for kernel density estimates
p_min = min(kde_p.support)
p_max = max(kde_p.support)
q_min = min(kde_q.support)
q_max = max(kde_q.support)
combined_min = min(p_min, q_min)
combined_max = max(p_max, q_max)

## Construct Sample from Multinomial Distribution

In [5]:
multinomial_sample_q = np.array([1, 2, 3, 2, 3, 3, 3, 2, 1, 1])
multinomial_sample_p = np.array([2, 2, 3, 2, 3])

In [6]:
# combined_sample = np.hstack((multinomial_sample_p, multinomial_sample_q))

In [7]:
# unique_combined = np.unique(combined_sample)
# unique_combined

In [8]:
# unique_q, counts_q = np.unique(multinomial_sample_q, return_counts=True)
# frequencies_q = counts_q / len(multinomial_sample_q)
# print(f'unique_q = {unique_q}')
# print(f'counts_q = {counts_q}')
# print(f'frequencies_q = {frequencies_q}')

In [9]:
# unique_p, counts_p = np.unique(multinomial_sample_p, return_counts=True)
# frequencies_p = counts_p / len(multinomial_sample_p)
# print(f'unique_p = {unique_p}')
# print(f'counts_p = {counts_p}')
# print(f'frequencies_p = {frequencies_p}')

In [10]:
# realization_to_frequency_dict_q = dict(zip(unique_q, frequencies_q))
# realization_to_frequency_dict_q

In [11]:
# realization_to_frequency_dict_p = dict(zip(unique_p, frequencies_p))
# realization_to_frequency_dict_p

In [12]:
# combined_frequencies_q = np.array([realization_to_frequency_dict_q.get(realization, 0.0) 
#                                    for realization 
#                                    in unique_combined])
# combined_frequencies_q

In [13]:
# combined_frequencies_p = np.array([realization_to_frequency_dict_p.get(realization, 0.0) 
#                                    for realization 
#                                    in unique_combined])
# combined_frequencies_p

In [14]:
# def discrete_relative_entropy_2(sample_p: np.ndarray, 
#                               sample_q: np.ndarray, 
#                               log_fun: tp.Callable = np.log):
#     combined_sample = np.hstack((sample_p, sample_q))
#     unique_combined = np.unique(combined_sample)
    
#     unique_q, counts_q = np.unique(sample_q, return_counts=True)
#     frequencies_q = counts_q / len(sample_q)
#     realization_to_frequency_dict_q = dict(zip(unique_q, frequencies_q))
    
#     unique_p, counts_p = np.unique(sample_p, return_counts=True)
#     frequencies_p = counts_p / len(sample_p)
#     realization_to_frequency_dict_p = dict(zip(unique_p, frequencies_p))
    
#     combined_frequencies_q = np.array([realization_to_frequency_dict_q.get(realization, 0.0) 
#                                        for realization 
#                                        in unique_combined])

#     combined_frequencies_p = np.array([realization_to_frequency_dict_p.get(realization, 0.0) 
#                                        for realization 
#                                        in unique_combined])

#     return sp.stats.entropy(pk=combined_frequencies_p, qk=combined_frequencies_q)
# #     return sp.stats.entropy(pk=combined_frequencies_q, qk=combined_frequencies_p)


# def construct_frequencies(sorted_p_realizations: np.ndarray, 
#                           sorted_p_frequencies: np.ndarray, 
#                           sorted_q_realizations: np.ndarray, 
#                           sorted_q_frequencies: np.ndarray, 
#                           sorted_combined_realizations: np.ndarray):
#     assert len(sorted_p_realizations) == len(sorted_p_frequencies)
#     assert len(sorted_q_realizations) == len(sorted_q_frequencies)

#     p_source_index = 0
#     q_source_index = 0
#     p_target_index = 0
#     q_target_index = 0
    
#     p_frequencies = np.zeros((len(sorted_p_realizations, )))
#     q_frequencies = np.zeros((len(sorted_p_realizations, )))
    
#     for combined_index in range(len(sorted_combined_realizations)):
#         realization = sorted_combined_realizations[combined_index]

#         print(f'combined_index = {combined_index}, realization = {realization}')
#         if sorted_p_realizations[p_source_index] != realization:
#             print(f'realization {realization} is not in p')
#             if sorted_q_realizations[q_source_index] == realization:
#                 print(f'but realization {realization} is in q')
#                 q_source_index += 1
#             continue
            
#         if sorted_p_frequencies[p_source_index] == 0.0:
#             p_source_index += 1
#             if sorted_q_realizations[q_source_index] == realization:
#                 q_source_index += 1
#             continue
            
#         if sorted_q_realizations[q_source_index] != realization or sorted_q_realizations[q_source_index] == 0.0:
#             if sorted_p_frequencies[p_source_index] != 0.0:   # we know that is true
#                 # if q(x) == 0 we must have p(x) == 0, which is not the case here
#                 raise ValueError('q(x) is zero but p(x) is not')
#             else:
#                 continue
        
#         p_frequencies[p_target_index] = sorted_p_frequencies[p_source_index]
#         q_frequencies[q_target_index] = sorted_q_frequencies[q_source_index]
#         p_source_index += 1
#         q_source_index += 1
#         p_target_index += 1
#         q_target_index += 1
            
#     return p_frequencies[:p_target_index], q_frequencies[:q_target_index]


# def discrete_relative_entropy(sample_p: np.ndarray, 
#                               sample_q: np.ndarray, 
#                               log_fun: tp.Callable = np.log):
#     combined_sample = np.hstack((sample_p, sample_q))
#     unique_combined = np.unique(combined_sample)
    
#     unique_q, counts_q = np.unique(sample_q, return_counts=True)
#     frequencies_q = counts_q / len(sample_q)
#     print(f'unique_q = {unique_q}')
    
#     unique_p, counts_p = np.unique(sample_p, return_counts=True)
#     frequencies_p = counts_p / len(sample_p)
#     print(f'unique_p = {unique_p}')
    
#     combined_frequencies_p, combined_frequencies_q = \
#         construct_frequencies(sorted_p_realizations=unique_p, 
#                               sorted_q_realizations=unique_q, 
#                               sorted_q_frequencies=frequencies_q, 
#                               sorted_p_frequencies=frequencies_p, 
#                               sorted_combined_realizations=unique_combined)
    
#     print(f'combined_frequencies_p = {combined_frequencies_p}')
#     print(f'combined_frequencies_q = {combined_frequencies_q}')
    
#     return np.sum(combined_frequencies_p * log_fun(combined_frequencies_p/combined_frequencies_q))

In [15]:
# discrete_relative_entropy_2(multinomial_sample_p, multinomial_sample_q)

In [16]:
discrete_relative_entropy(multinomial_sample_p, multinomial_sample_q)

combined_index = 0, realization = 1
realization 1 is not in p
but realization 1 is in q
combined_index = 1, realization = 2
combined_index = 2, realization = 3


0.4158883083359672

In [17]:
# discrete_relative_entropy(multinomial_sample_q, multinomial_sample_p)

In [18]:
# def _counts(sample: np.ndarray):
#     _, counts = np.unique(sample, return_counts=True)
#     return counts


# def _frequencies(sample: np.ndarray):
#     return _counts(sample) / len(sample)


# def discrete_entropy(sample: np.ndarray, log_fun: tp.Callable = np.log):
#     frequencies = _frequencies(sample)
#     return - np.sum(frequencies * log_fun(frequencies))

In [19]:
discrete_entropy(multinomial_sample_q, log_fun=np.log2)

1.5709505944546684

In [20]:
discrete_entropy(multinomial_sample_p, log_fun=np.log2)

0.9709505944546686

In [21]:
discrete_entropy(multinomial_sample_p, log_fun=np.log2)

0.9709505944546686

In [22]:
sp.stats.entropy(_frequencies(multinomial_sample_q), base=2)

NameError: name '_frequencies' is not defined

In [ ]:
sp.stats.entropy(_frequencies(multinomial_sample_p), base=2)

# Entropy

The entropy of a probability distribution $p$ is defined as 

$H(X) = - \mathbb{E}_p \left[ \log_{\text{base}} p \right]$, 

where $\mathbb{E}_P$ denotes expectation with respect the probability distribution $p$. In information theory, the base of the logarithm is 2 and the interpretation of entropy is the average number of bits needed to optimally encode the signal represented by the distribution $p$. 

Divergence defaults to $\text{base}=e$, which results in the natural logarithm i.e. $\log_e = \ln$. This default choice can be overridden by specifying a different logarithmic function than the natural logarithm in the argument 'log_fun' during the entropy calculation. In particular, specifying $\text{base}=2$ by setting 'log_fun=np.log2' results in the classical Shannon entropy expressed in bits, whereas specifying $\text{base}=10$ by setting 'log_fun=np.log10' produces the entropy in decimal bits (dits or Hartleys).

## Entropy from Statsmodels KDE Objects (via Statsmodels)

In [ ]:
print(f'Entropy of p = {kde_p.entropy}')
print(f'Entropy of q = {kde_q.entropy}')

## Entropy from Statsmodels KDE Objects (via Divergence)

In [ ]:
print(f'Entropy of p = {compute_entropy_from_kde(kde_p)}')
print(f'Entropy of q = {compute_entropy_from_kde(kde_q)}')

## Entropy from Normal Probability Density Functions

In [ ]:
print(f'Entropy of p = {compute_entropy_from_density_with_support(pdf_p, p_min, p_max)}')
print(f'Entropy of q = {compute_entropy_from_density_with_support(pdf_q, q_min, q_max)}')

## Theoretical Entropy of a Normal Distribution

In [ ]:
def theoretical_entropy_of_normal_distribution(mu: float, sigma: float, log_fun: tp.Callable = np.log) -> float:
    return 0.5 * (1.0 + log_fun(2 * np.pi * sigma**2))

print(f'Entropy of p = {theoretical_entropy_of_normal_distribution(mu_p, sigma_p)}')
print(f'Entropy of q = {theoretical_entropy_of_normal_distribution(mu_q, sigma_q)}')

# Cross Entropy

The cross entropy of a distribution $q$ relative to a distribution $p$ is defined as  

$H(p, q) = - \mathbb{E}_p \left[ \log_{\text{base}} q \right]$.

With a base of 2, the cross-entropy of $q$ relative to $p$ is the average number of bits required to encode the signal in $p$ using a code optimized for the signal in $q$.

## Cross Entropy from Statsmodels KDE Objects

In [ ]:
print(f'Cross Entropy of p relative to q = {compute_cross_entropy_from_kde(kde_p, kde_q)}')
print(f'Cross Entropy of q relative to p = {compute_cross_entropy_from_kde(kde_q, kde_p)}')

## Cross Entropy from Normal Probability Density Functions

In [ ]:
print(f'Cross Entropy of p relative to q = {compute_cross_entropy_from_densities_with_support(pdf_p, pdf_q, combined_min, combined_max)}')
print(f'Cross Entropy of q relative to p = {compute_cross_entropy_from_densities_with_support(pdf_q, pdf_p, combined_min, combined_max)}')

# Relative Entropy (Kullback-Leibler Divergence)

Relative entropy or Kullback-Leibler divergence measures the dispersion of two probability distributions $P$ and $Q$. It is defined as the difference between the cross entropy of $q$ relative to $p$ and the entropy of $p$

$D_{KL} (P||Q) = \mathbb{E}_p \left[ \log_{\text{base}} \left( \frac{p}{q} \right) \right] = H(p, q) - H(p)$.

With a base of 2, it can be interpreted as the average number of additional bits required to encode the signal in $p$ using a code optimized for the signal in $q$ over and above the number of bits required by the optimal code for $p$.

## Relative Entropy from Statsmodels KDE Objects

In [ ]:
print(f'Relative Entropy of p relative to q = {compute_relative_entropy_from_kde(kde_p, kde_q)}')
print(f'Relative Entropy of q relative to p = {compute_relative_entropy_from_kde(kde_q, kde_p)}')

## Relative Entropy from Normal Probability Density Functions

In [ ]:
print(f'Relative Entropy from p to q = {compute_relative_entropy_from_densities_with_support(pdf_p, pdf_q, combined_min, combined_max)}')
print(f'Relative Entropy from q to p = {compute_relative_entropy_from_densities_with_support(pdf_q, pdf_p, combined_min, combined_max)}')

## Theoretical Relative Entropy for Normal Distributions

In [ ]:
def relative_entropy_between_normal_distributions(mu_1, sigma_1, mu_2, sigma_2, log_fun: tp.Callable = np.log):
    return ((mu_1 - mu_2)**2 + sigma_1**2 - sigma_2**2 ) / (2 * sigma_2**2) + log_fun(sigma_2/sigma_1)

print(f'Relative Entropy from p to q = {relative_entropy_between_normal_distributions(mu_p, sigma_p, mu_q, sigma_q)}')
print(f'Relative Entropy from q to p = {relative_entropy_between_normal_distributions(mu_q, sigma_q, mu_p, sigma_p)}')

# Jensen-Shannon Divergence

The Jensen-Shannon divergence, a symmetric measure of the divergence of probability distributions, is defined as

$JSD(p||q) = \frac{1}{2} D_{KL} (p||m) + \frac{1}{2} D_{KL} (q||m)$, 

where $m = \frac{1}{2} \left( p + q \right)$.

For base 2, the JSD is bounded between 0 and 1. For base $e$, it is bounded between $0$ and $\ln(2)$.

## Jensen-Shannon Divergence from Statsmodels KDE Objects

In [ ]:
print(f'Jensen-Shannon Divergence between p and q = {compute_jensen_shannon_divergence_from_kde(kde_p, kde_q)}')
print(f'Jensen-Shannon Divergence between q and p = {compute_jensen_shannon_divergence_from_kde(kde_q, kde_p)}')

## Jensen-Shannon Divergence from Normal Probability Density Functions

In [ ]:
print(f'Jensen-Shannon Divergence between p and q = {compute_jensen_shannon_divergence_from_densities_with_support(pdf_p, pdf_q, combined_min, combined_max)}')
print(f'Jensen-Shannon Divergence between q and p = {compute_jensen_shannon_divergence_from_densities_with_support(pdf_q, pdf_p, combined_min, combined_max)}')

## Jensen-Shannon Divergence from Statsmodels KDE Objects in Bits

In [ ]:
print(f'Jensen-Shannon Divergence between p and q = {compute_jensen_shannon_divergence_from_kde(kde_p, kde_q, log_fun=np.log2)}')
print(f'Jensen-Shannon Divergence between q and p = {compute_jensen_shannon_divergence_from_kde(kde_q, kde_p, log_fun=np.log2)}')